In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

In [ ]:
os.chdir('/content/gdrive/My Drive')

In [ ]:
file_path = os.getcwd() + '/1'

In [ ]:
# file_ = open(file_path , 'r' , encoding='iso-8859-1')
# 
# data = file_.readlines()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

# from faker import Faker
import random
from tqdm import tqdm
# from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import re
import unicodedata
import urllib3
import shutil
import zipfile
import itertools
from string import digits

2.2.0


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def preprocess_sentence(sentence):
    
    num_digits= str.maketrans('','', digits)
    
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    sentence= sentence.translate(num_digits)
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence = "<start> " + sentence + " <end>"
    
    return sentence


In [ ]:

text = []
count = 0


with open(file_path ,'r' , encoding='iso-8859-1') as txtfile:
  for line in txtfile.readlines():
    if count == 20000:
      break

    text.append(preprocess_sentence(line.split('+++$+++')[-1]))
    count +=1
    
    


In [ ]:
len(text)

20000

In [ ]:
text[0]

'<start> they do not ! <end>'

In [ ]:

max_len = 20
trunc_token = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(oov_token=oov_tok , filters = "")
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text)
padded = pad_sequences(sequences,maxlen=max_len, truncating=trunc_token,padding = 'post')


In [ ]:
len(word_index)

13051

In [ ]:
# padded[10008]

In [ ]:
index_word = {}
for word , index in word_index.items():
    index_word[index] = word
    
    

In [ ]:
# !unzip 1835_3176_compressed_glove.6B.200d.txt.zip

In [ ]:
os.listdir()

['glove.6B.200d.txt',
 'Colab Notebooks',
 'data',
 '1835_3176_compressed_glove.6B.200d.txt.zip',
 '1',
 'cpoint']

In [ ]:
def create_word_embeddings(file_path):
    
    with open(file_path , 'r') as f:
        wordToEmbedding = {}
        wordToIndex = {}
        indexToWord = {}
        
        for line in f:
            data = line.strip().split()
            token = data[0]
            wordToEmbedding[token] = np.array(data[1:] ,dtype = np.float64)



        
        tokens = sorted(wordToEmbedding.keys())
        for idx , token in enumerate(tokens):
            idx = idx + 1 #for zero masking
            wordToIndex[token] = idx
            indexToWord[idx] = token

    return wordToEmbedding , wordToIndex , indexToWord
            
            

In [ ]:
wordToEmbedding , wordToIndex , indexToWord = create_word_embeddings(os.getcwd()+ '/glove.6B.200d.txt')

In [ ]:
len(wordToIndex)    

400000

In [ ]:
def create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord):
    
    vocablen = len(word_index)+1 #for zero masking
    embedding_dimensions = 200
    
    embeddingMatrix = np.zeros((vocablen , embedding_dimensions))
    count = 0
    for word , index in word_index.items():
        if word not in wordToEmbedding.keys():
            embeddingMatrix[index ,:] = np.random.uniform(low = -1 , high =1 ,size = (1,200))
            count +=1
        else :
            
            embeddingMatrix[index , :] = wordToEmbedding[word]
        
    embeddingLayer = Embedding(vocablen , embedding_dimensions , weights = [embeddingMatrix] , trainable = False)
    print(embeddingMatrix.shape)
    print(count)
    
    return embeddingMatrix

In [ ]:
len(word_index)

13051

In [ ]:
embeddingMatrix = create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord)


(13052, 200)
1601


In [ ]:
embeddingMatrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5951225 , -0.58297815,  0.64335446, ...,  0.02737674,
         0.94545301,  0.41325264],
       [ 0.12289   ,  0.58037   , -0.069635  , ..., -0.039174  ,
        -0.16236   , -0.096652  ],
       ...,
       [-0.25924   , -0.81193   ,  0.45847   , ...,  0.045301  ,
         0.01518   , -0.36259   ],
       [ 0.53246   ,  0.84359   ,  0.13337   , ...,  0.11501   ,
        -0.33268   , -0.55189   ],
       [-0.018711  ,  0.39573   ,  0.84356   , ...,  0.31973   ,
        -0.28322   , -0.12453   ]])

In [ ]:
def get_dataset(padded):
    index = 0
    context = np.zeros((1000,max_len))
    response = np.zeros((1000,max_len))
    for idx in range(2000,4000,2):
        context[index,:] = padded[idx]
        response[index,:] = padded[idx+1]
        index +=1
    return context , response

In [ ]:
context , response = get_dataset(padded)
BATCH_SIZE = 100
BUFFER_SIZE = len(context)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 200
rnn_units = 1024
dense_units = 1024
Dtype = tf.float32   #used to initialize DecoderCell Zero state


In [ ]:
response

array([[   3.,   57.,   27., ...,    0.,    0.,    0.],
       [   3.,   87.,  108., ...,    0.,    0.,    0.],
       [   3., 1818.,    4., ...,    0.,    0.,    0.],
       ...,
       [   3.,   20.,   27., ...,    0.,    0.,    0.],
       [   3.,  310.,    7., ...,    0.,    0.,    0.],
       [   3.,  364.,    2., ...,    0.,    0.,    0.]])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((context, response)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape) 
print(example_Y.shape)

(100, 20)
(100, 20)


In [ ]:
Tx = 20
Ty = 20
vocab_size = len(word_index) + 1

In [ ]:
class EncoderNetwork(tf.keras.Model):
    def __init__(self,input_vocab_size,embedding_dims, rnn_units ,embeddings_matrix):
        super().__init__()
        self.encoder_embedding = tf.keras.layers.Embedding(input_dim = input_vocab_size,
                                                       output_dim = embedding_dims,
                                                       weights = [embeddings_matrix], trainable = False)
        self.encoder_rnnlayer = tf.keras.layers.LSTM(rnn_units,return_sequences=True, 
                                                     return_state=True )
    
#DECODER
class DecoderNetwork(tf.keras.Model):
    def __init__(self,output_vocab_size, embedding_dims, rnn_units , embeddings_matrix):
        super().__init__()
        self.decoder_embedding = tf.keras.layers.Embedding(input_dim = output_vocab_size,
                                                       output_dim = embedding_dims,
                                                       weights = [em/beddings_matrix], trainable = False) 
        self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
        self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell =  self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler= self.sampler,
                                                output_layer=self.dense_layer)

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.LuongAttention(units, memory = memory, 
                                          memory_sequence_length=memory_sequence_length)
        #return tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)

    # wrap decodernn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state,Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 
        return decoder_initial_state



encoderNetwork = EncoderNetwork(len(word_index)+1,embedding_dims, rnn_units,embeddingMatrix)
decoderNetwork = DecoderNetwork(len(word_index)+1,embedding_dims, rnn_units , embeddingMatrix)

In [ ]:
print(len(word_index))

13051


In [ ]:
# index_word[382]

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
def loss_function(y_pred, y):
   
    #shape of y [batch_size, ty]
    #shape of y_pred [batch_size, Ty, output_vocab_size] 
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    #skip loss calculation for padding sequences i.e. y = 0 
    #[ <start>,How, are, you, today, 0, 0, 0, 0 ....<end>]
    #[ 1, 234, 3, 423, 3344, 0, 0 ,0 ,0, 2 ]
    # y is a tensor of [batch_size,Ty] . Create a mask when [y=0]
    # mask the loss when padding sequence appears in the output sequence
    mask = tf.logical_not(tf.math.equal(y,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
decoderNetwork.attention_mechanism.memory_initialized

False

In [ ]:

def train_step(input_batch, output_batch,encoder_initial_cell_state):
    #initialize loss = 0
    loss = 0
    with tf.GradientTape() as tape:
        encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
        a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                        initial_state =encoder_initial_cell_state)

        #[last step activations,last memory_state] of encoder passed as input to decoder Network
        
         
        # Prepare correct Decoder input & output sequence data
        decoder_input = output_batch[:,:-1] # ignore <end>
        #compare logits with timestepped +1 version of decoder_input
        decoder_output = output_batch[:,1:] #ignore <start>


        # Decoder Embeddings
        decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

        #Setting up decoder memory from encoder output and Zero State for AttentionWrapperState
        decoderNetwork.attention_mechanism.setup_memory(a)
        decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                           encoder_state=[a_tx, c_tx],
                                                                           Dtype=tf.float32)
        
        #BasicDecoderOutput        
        outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                               sequence_length=BATCH_SIZE*[Ty-1])

        logits = outputs.rnn_output
        #Calculate loss

        loss = loss_function(logits, decoder_output)

    #Returns the list of all layer variables / weights.
    variables = encoderNetwork.trainable_variables + decoderNetwork.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss

In [ ]:
checkpointdir = os.path.join('/content/gdrive/My Drive/',"cpoint")
chkpoint_prefix = os.path.join(checkpointdir, "cpoint")
if not os.path.exists(checkpointdir):
    os.mkdir(checkpointdir)

checkpoint = tf.train.Checkpoint(optimizer = optimizer, encoderNetwork = encoderNetwork, 
                                 decoderNetwork = decoderNetwork)

try:
    status = checkpoint.restore(tf.train.latest_checkpoint(checkpointdir))
    print("Checkpoint found at {}".format(tf.train.latest_checkpoint(checkpointdir)))
except:
    print("No checkpoint found at {}".format(checkpointdir))

Checkpoint found at None


In [ ]:
def initialize_initial_state():
        return [tf.zeros((BATCH_SIZE, rnn_units)), tf.zeros((BATCH_SIZE, rnn_units))]

In [ ]:
epochs = 100
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0


    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%2 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))
            checkpoint.save(file_prefix = chkpoint_prefix)

total loss: 0.15453994274139404 epoch 1 batch 2 
total loss: 0.1342565417289734 epoch 1 batch 4 
total loss: 0.17941759526729584 epoch 1 batch 6 
total loss: 0.1647564172744751 epoch 1 batch 8 
total loss: 0.14172464609146118 epoch 1 batch 10 
total loss: 0.13640350103378296 epoch 2 batch 2 
total loss: 0.15635256469249725 epoch 2 batch 4 
total loss: 0.16196973621845245 epoch 2 batch 6 
total loss: 0.16154317557811737 epoch 2 batch 8 
total loss: 0.1764344424009323 epoch 2 batch 10 
total loss: 0.11377259343862534 epoch 3 batch 2 
total loss: 0.13172246515750885 epoch 3 batch 4 
total loss: 0.1238851472735405 epoch 3 batch 6 
total loss: 0.13810159265995026 epoch 3 batch 8 
total loss: 0.13879939913749695 epoch 3 batch 10 
total loss: 0.11919151991605759 epoch 4 batch 2 
total loss: 0.1417405903339386 epoch 4 batch 4 
total loss: 0.10600625723600388 epoch 4 batch 6 
total loss: 0.1147468164563179 epoch 4 batch 8 


KeyboardInterrupt: ignored

In [ ]:
decoder_embedding_matrix = decoderNetwork.decoder_embedding.variables[0] 
print(decoderNetwork.decoder_embedding.variables[0].shape)

(13052, 200)


In [ ]:
beam_width = 3
#use with scope /cpu:0 for inferencing
#restore from latest checkpoint for inferencing
input_raw=" how was your day\n who are you \n do you support congress? \n easy way to die \n find a girl for me \n When will this project get over? "
#input_raw="Wow!"  #checking translation on training set record
#def inference(input_raw):
input_lines = input_raw.split("\n")
# We have a transcript file containing English-Hindi pairs
# Preprocess X
input_lines = [preprocess_sentence(line) for line in input_lines]
input_sequences = [[word_index[w] for w in line.split(' ')] for line in input_lines]
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                maxlen=Tx, padding='post')
inp = tf.convert_to_tensor(input_sequences)
#print(inp.shape)
inference_batch_size = input_sequences.shape[0]
encoder_initial_cell_state = [tf.zeros((inference_batch_size, rnn_units)),
                              tf.zeros((inference_batch_size, rnn_units))]
encoder_emb_inp = encoderNetwork.encoder_embedding(inp)
a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp,
                                                initial_state =encoder_initial_cell_state)

start_tokens = tf.fill([inference_batch_size],word_index['<start>'])
#print(start_tokens)
end_token = word_index['<end>']



decoder_input = tf.expand_dims([word_index['<start>']]* inference_batch_size,1)
decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)


#From official documentation
#NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:

#The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
#The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
#The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.
encoder_memory = tfa.seq2seq.tile_batch(a, beam_width)
decoderNetwork.attention_mechanism.setup_memory(encoder_memory)
print("beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] :", encoder_memory.shape)
#set decoder_inital_state which is an AttentionWrapperState considering beam_width
decoder_initial_state = decoderNetwork.rnn_cell.get_initial_state(batch_size = inference_batch_size* beam_width,dtype = Dtype)
encoder_state = tfa.seq2seq.tile_batch([a_tx, c_tx], multiplier=beam_width)
decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 

decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoderNetwork.rnn_cell,beam_width=beam_width,
                                                 output_layer=decoderNetwork.dense_layer)


# Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
# One heuristic is to decode up to two times the source sentence lengths.
maximum_iterations = tf.round(tf.reduce_max(Tx) * 2)

#initialize inference decoder

(first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_token,
                             initial_state = decoder_initial_state)
#print( first_finished.shape)
print("\nfirst_inputs returns the same decoder_input i.e. embedding of  <start> :",first_inputs.shape)

inputs = first_inputs
state = first_state  
predictions = np.empty((inference_batch_size, beam_width,0), dtype = np.int32)
beam_scores =  np.empty((inference_batch_size, beam_width,0), dtype = np.float32)                                                                            
for j in range(maximum_iterations):
    beam_search_outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
    inputs = next_inputs
    state = next_state
    outputs = np.expand_dims(beam_search_outputs.predicted_ids,axis = -1)
    scores = np.expand_dims(beam_search_outputs.scores,axis = -1)
    predictions = np.append(predictions, outputs, axis = -1)
    beam_scores = np.append(beam_scores, scores, axis = -1)
print(predictions.shape) 
print(beam_scores.shape)


beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (18, 20, 256)

first_inputs returns the same decoder_input i.e. embedding of  <start> : (6, 3, 200)
(6, 3, 40)
(6, 3, 40)


In [ ]:
for i in range(len(predictions)):
    print("---------------------------------------------")
    output_beams_per_sample = predictions[i,:,:]
    score_beams_per_sample = beam_scores[i,:,:]
    for beam, score in zip(output_beams_per_sample,score_beams_per_sample) :
        seq = list(itertools.takewhile( lambda index: index !=2, beam))
        score_indexes = np.arange(len(seq))
        beam_score = score[score_indexes].sum()
        print(" ".join( [index_word[w] for w in seq]), " beam score: ", beam_score)


---------------------------------------------
let ,  beam score:  -4.7131248
sure join when dont you call ! man  beam score:  -33.260674
make it to was was an , <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>  beam score:  -254.03714
---------------------------------------------
whyd you -- to is my wife  beam score:  -19.977266
no  beam score:  -1.839639
someone like its not someone  beam score:  -15.914265
---------------------------------------------
i dont know  beam score:  -1.503685
you know ? <end> <end> <end> take could  beam score:  -20.578503
im dont me , you thought i start  beam score:  -27.61116
---------------------------------------------
you not anything  beam score:  -5.2303247
youre didnt <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <e